In [1]:
import pandas as pd
import numpy as np
import statistics as st
from sklearn.ensemble import RandomForestClassifier

In [2]:
DATA_PATH = r"../../../../../Data/Elo Merchant Category Recommendation/"

In [3]:
test = pd.read_csv(DATA_PATH + "test.csv", header = "infer", sep = ",")
#(123623, 5)
# first_active_month, card_id, feature_1, feature_2, feature_3

In [4]:
test1 = test.copy() # substitute NaN with the mode

In [5]:
test2 = test.copy() #substitute NaN with mean

In [6]:
test3 = test.copy() #substitute NaN with kmeans cluster center

In [7]:
test.loc[:,"feature_3"].value_counts(dropna=False)

1    69770
0    53853
Name: feature_3, dtype: int64

In [8]:
test.loc[test.isnull().any(axis=1),:]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaN,C_ID_c27b4f80f7,5,2,1


**test1**
 - assign NaN row "first_active_month" the mod value

In [9]:
test1.loc[test.isnull().any(axis=1),"first_active_month"] = "2017-09"

In [10]:
test1["first_active_year"] = test1.loc[:,"first_active_month"].apply(lambda x: int(str(x).split("-")[0]))

In [11]:
test1["first_active_month"] = test1.loc[:,"first_active_month"].apply(lambda x: int(str(x).split("-")[1]))

In [12]:
test1.to_csv(DATA_PATH + "test1.csv")

**test2**
 - assign NaN row "first_active_month" the "0-0" value
 - extract year and month
 - assign mean values to the NaN row

In [13]:
test2.loc[test.isnull().any(axis=1),"first_active_month"] = "0-0"

In [14]:
test2["first_active_year"] = test2.loc[:,"first_active_month"].apply(lambda x: int(str(x).split("-")[0]))

In [15]:
test2["first_active_month"] = test2.loc[:,"first_active_month"].apply(lambda x: int(str(x).split("-")[1]))

In [16]:
test2.loc[test2["first_active_year"] == 0,"first_active_year"] = int(st.mean(test2.loc[test2["first_active_year"] != 0,"first_active_year"]))

In [17]:
test2.loc[test2["first_active_month"] == 0,"first_active_month"] = int(st.mean(test2.loc[test2["first_active_month"] != 0,"first_active_month"]))

In [18]:
test2.to_csv(DATA_PATH + "test2.csv")

**test3**
 - select all rows besides NaN row "first_active_month"
 - run random forests
 - predict NaN value and substitute
 - extract year and month

In [19]:
tr_part = test3[test3.first_active_month.notnull()]

In [20]:
x_train = tr_part.loc[:,["feature_1","feature_2","feature_3"]]

In [21]:
y_train = tr_part.loc[:,"first_active_month"]

In [22]:
x_test = test3[test3.first_active_month.isnull()].loc[:,["feature_1","feature_2","feature_3"]]

In [23]:
model = RandomForestClassifier(n_estimators = 42)

In [24]:
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=42, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
y_test = model.predict(x_test)

In [26]:
test3.loc[test3.isnull().any(axis = 1),"first_active_month"] = y_test

In [27]:
test3["first_active_year"] = test3.loc[:,"first_active_month"].apply(lambda x: int(str(x).split("-")[0]))

In [28]:
test3["first_active_month"] = test3.loc[:,"first_active_month"].apply(lambda x: int(str(x).split("-")[1]))

In [29]:
test3.to_csv(DATA_PATH + "test3.csv")